In [2]:
import os, sys, shutil
import rasterio
import numpy as np
import geopandas as gpd
import pandas as pd
import tqdm


from glob import glob
from rasterio.warp import transform_bounds
from shapely.geometry import Polygon
from pyproj import Transformer
from matplotlib import pyplot as plt

module_path = os.path.abspath("../..") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)

from dataset import load_scenario
from Scenario_sampler import ScenarioSampler

In [4]:
# utils

def return_first_scenario(path):
    chosen_subfolder = None
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_dir() and entry.name != '.DS_Store':
                chosen_subfolder = entry.name
                break
    if chosen_subfolder is None:
        raise ValueError("No subfolder found")
    return chosen_subfolder



def read_first_and_last_lines(file_path):
    with open(file_path, 'rb') as file:
        # Read the first line
        first_line = file.readline().decode()

        # Move to the end of the file
        file.seek(0, os.SEEK_END)
        file_size = file.tell()
        if file_size == 0:
            return '', ''  # Empty file

        pos = file_size - 1

        # Skip trailing newlines at the end of the file
        while pos >= 0:
            file.seek(pos, os.SEEK_SET)
            char = file.read(1)
            if char != b'\n' and char != b'\r':
                break
            pos -= 1

        # Now find the previous newline (start of last line)
        while pos >= 0:
            file.seek(pos, os.SEEK_SET)
            if file.read(1) == b'\n':
                pos += 1
                break
            pos -= 1
        else:
            pos = 0  # The last line is also the first line

        file.seek(pos, os.SEEK_SET)
        last_line = file.readline().decode()

    return first_line, last_line


def read_first_and_last_lines2(file_path):
    with open(file_path, 'r') as file:
        # Read the first line
        first_line = file.readline()

        # Move to the end of the file
        file.seek(-1, 2)  # 2 means seek from the end

        # Read until a newline is found
        last_line = ''
        while True:
            char = file.read(1)
            if char == '\n' or file.tell() == 1:
                break
            last_line = char + last_line
            file.seek(-2, 1)  # Move back two characters

    return first_line, last_line

from datetime import datetime
def find_earliest_latest_dates(layout_path):
    """
    Find the earliest and latest dates in a layout folder.
    """
    earliest_date = None
    latest_date = None
    for filename in os.listdir(os.path.join(layout_path, "Weather_Data")):
        if filename.endswith(".txt"):
            first_line, last_line = read_first_and_last_lines(os.path.join(layout_path, "Weather_Data", filename))
            first_date = " ".join(first_line.split(" ")[:4])
            last_date = " ".join(last_line.split(" ")[:4])
            first_date = datetime.strptime(first_date, "%Y %m %d %H%M")
            last_date = datetime.strptime(last_date, "%Y %m %d %H%M")
            if earliest_date is None or first_date < earliest_date:
                earliest_date = first_date
            if latest_date is None or last_date > latest_date:
                latest_date = last_date
    return earliest_date, latest_date

In [3]:
new_fires_gdf = gpd.read_file("./newfires.gpkg") # 6th edition # gpd.read_file("./FPA_FOD_20210617.gpkg") 5th edition  
# replace illegal dates by jan first 1900
new_fires_gdf['DISCOVERYDATETIME'] = new_fires_gdf['DISCOVERYDATETIME'].replace('1001/01/01 00:00:00+00', '1900/01/01 00:00:00+00')
new_fires_gdf['DISCOVERY_DATE'] = pd.to_datetime(new_fires_gdf['DISCOVERYDATETIME'])
new_fires_gdf['DISCOVERY_DATE'].dt.date
new_fires_gdf['LONGITUDE'] = new_fires_gdf['LONGDD83']
new_fires_gdf['LATITUDE'] = new_fires_gdf['LATDD83']
new_fires_gdf = new_fires_gdf.to_crs("EPSG:4326")
print(len(new_fires_gdf)) # 

old_fires_gdf = gpd.read_file("./FPA_FOD_20221014.gpkg")
old_fires_gdf['OBJECTID'] = old_fires_gdf['FOD_ID']
old_fires_gdf = old_fires_gdf.to_crs("EPSG:4326")
print(len(old_fires_gdf)) # 2 303 566

# merge the two gdfs
fires_gdf = pd.concat([new_fires_gdf, old_fires_gdf])
print(len(fires_gdf))

# drop duplicates as defined as same lat, long and same date
fires_gdf = fires_gdf.drop_duplicates(subset=['LATITUDE', 'LONGITUDE', 'DISCOVERY_DATE'])
print(len(fires_gdf)) 

# fires_gdf = new_fires_gdf.to_crs("EPSG:4326")
# print(len(fires_gdf))

582291


/opt/anaconda3/envs/wildfire/lib/python3.11/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'FPA_FOD_20221014.gpkg': 'Fires' (default), 'NWCG_UnitIdActive_20200123'. Specify layer parameter to avoid this warning.
  result = read_func(


2303566
2885857
2843606


In [11]:
new_fires_gdf.iloc[0]['DISCOVERY_DATE'].strftime('%m-%d')

'07-03'

In [5]:
# Get the layout coordinates 
# 1 extract the tif files
dataset_path = "./WideDataset/"

# copy the layout tifs from a folder to a folder called sim2real_layout
tif_path = "sim2real_layouts"
os.makedirs(tif_path, exist_ok=True)
for folder in os.listdir(dataset_path):
    if folder == ".DS_Store":
        continue
    shutil.copy(f"WideDataset/{folder}/Vegetation_Map/Existing_Vegetation_Cover.tif", f"{tif_path}/{folder}_Existing_Vegetation_Cover.tif")

print("Copied the layout tifs to sim2real_layouts")
####

tif_files = glob(os.path.join(tif_path, "*.tif"))
layout_list = []
widths = []
names = []
for tif_file in tif_files:
    with rasterio.open(tif_file) as dataset:
        # get the file name without the path
        file_name = os.path.basename(tif_file)  # '0004_Elevation.tif'
        identifier = "_".join(file_name.split('_')[:2])

        # extract the resolution, check it is 30
        x_resolution = dataset.transform[0]
        y_resolution = -dataset.transform[4]
        assert x_resolution == y_resolution == 30, f"Resolution is not the same: {x_resolution} != {y_resolution}"

        # extract the coordinates using the bounds
        # /!\ DO NOT USE transform_bounds(dataset.crs, 'EPSG:4326', *dataset.bounds)
        x_min = dataset.bounds[0]
        x_max = dataset.bounds[2]
        y_min = dataset.bounds[1]
        y_max = dataset.bounds[3]

        transformer = Transformer.from_crs(dataset.crs, "EPSG:4326", always_xy=True)
        lat_top_left, lon_top_left = transformer.transform(x_min, y_max)
        lat_top_right, lon_top_right = transformer.transform(x_max, y_max)
        lat_bottom_left, lon_bottom_left = transformer.transform(x_min, y_min)
        lat_bottom_right, lon_bottom_right = transformer.transform(x_max, y_min)

        # Create the polygon using the transformed bounds
        polygon = Polygon((
            (lat_top_left, lon_top_left),
            (lat_top_right, lon_top_right),
            (lat_bottom_right, lon_bottom_right),
            (lat_bottom_left, lon_bottom_left),
            (lat_top_left, lon_top_left)  # close the polygon
        ))

        layout_list.append({
            'identifier': identifier,
            'height': dataset.height,
            'width': dataset.width,
            'geometry': polygon,
            'transformer': transformer,
            'dataset': dataset
        })


        widths.append(dataset.width)
        names.append(identifier)

sorted_indices = np.argsort(widths)
widths = np.array(widths)[sorted_indices]
names = np.array(names)[sorted_indices]
filtered_layout_list = [layout_list[i] for i in sorted_indices]

n_small_layouts = len(widths[widths < 500])
n_medium_layouts = len(widths[(widths >= 500) & (widths < 1000)])
n_large_layouts = len(widths[widths >= 1000])

small_layouts = filtered_layout_list[:n_small_layouts]
medium_layouts = filtered_layout_list[n_small_layouts:n_small_layouts + n_medium_layouts]
large_layouts = filtered_layout_list[n_small_layouts + n_medium_layouts:]

selected_layouts = medium_layouts + large_layouts
widths = widths[-len(selected_layouts):]
names = names[-len(selected_layouts):]
print(max(widths))
raise Exception("stop")


print("loaded the layout list")
# move the samll layouts to the small folder
os.makedirs("small_layouts", exist_ok=True)
for layout in small_layouts:
    #print(os.path.join(dataset_path, layout['identifier']), os.path.join("small_layouts", layout['identifier']))
    try:
        shutil.move(os.path.join(dataset_path, layout['identifier']), os.path.join("small_layouts", layout['identifier']))
    except Exception as e:
       # file already moved
       pass


# convert the layout list to a geopandas df
layout_list = selected_layouts
gdf = gpd.GeoDataFrame(layout_list, geometry='geometry', crs="EPSG:4326")




# Load the historical fires 
# fires_gdf = gpd.read_file("./FPA_FOD_20210617.gpkg")
# fires_gdf = fires_gdf.to_crs("EPSG:4326")
print("loaded the fires")


# Joint
# Spatial join: find which points fall into which polygons
joined = gpd.sjoin(fires_gdf, gdf, how='inner', predicate='within')


# Count points per polygon
counts = joined.groupby('identifier').size().reset_index(name='fire_count')


dataset_path = "./WideDataset/"
scenario_path_suffix = "/Satellite_Images_Mask/"

failed_layouts = []
continue_out = False
processed = total = 0

for layout_name in tqdm.tqdm(names):
    total+=1
    print(layout_name)
    try:
        layout_folder = dataset_path + layout_name + scenario_path_suffix
        # check the layout folder exists
        if not os.path.exists(layout_folder):
            print(f"Layout {layout_folder} does not exist")
            layout_folder = dataset_path + layout_name +  "/Satellite_Image_Mask/"
            if not os.path.exists(layout_folder):
                print(f"Layout {layout_folder} does not exist")
                failed_layouts.append(layout_name)
                continue
        
        if os.path.exists(f"./WideDataset/{layout_name}/selected_scenarios_full_match.txt"):
            processed +=1
            #continue

        # check that the scenario have the right size
        first_scenario = return_first_scenario(layout_folder)
        if first_scenario is None:
            print(f"Layout {layout_name} does not have any scenario")
            failed_layouts.append(layout_name)
            continue
        
        first_loaded_scenario = load_scenario(os.path.join(layout_folder, first_scenario), extension = '.jpg', first_frame_only=True)
        height_scenario, width_scenario = first_loaded_scenario.shape[0], first_loaded_scenario.shape[1]

        data = joined[joined['identifier'] == layout_name]
        if len(data) == 0:
            print(f"No fires found for layout {layout_name}")
            failed_layouts.append(layout_name)
            continue
        plt.scatter(data["LONGITUDE"], data["LATITUDE"])
        plt.axis("equal")
        plt.show()

        #earliest_date, latest_date = find_earliest_latest_dates(dataset_path + layout_name)
        #print(f"Earliest date: {earliest_date}, Latest date: {latest_date}")


        sampler = ScenarioSampler(layout_folder, extension = '.jpg')
        sampled_scenarios = []
        sampled_ignition_points = []
        associated_fires = []
        date_matched = []
        distances = []
        failed = 0
        # plot the historical fires
        

        # start with the fires that have potewntial to be test fires, i.e their date is between the earliest and latest date
        # We will have one "test" dataset, one "train" dataset, and one extra train dataset for the test fires
        # filtered_data = data[
        # (data['DISCOVERY_DATE'].dt.date >= earliest_date.date()) & 
        # (data['DISCOVERY_DATE'].dt.date <= latest_date.date())
        # ]
        

        for i, fire in data.iterrows():
            # print the coordinates of the fire
            width, height = fire['width'], fire['height']
            # check that the scenario have the right size
            if width != width_scenario or height != height_scenario:
                print(f"Scenario {first_scenario} has the wrong size for layout {layout_name}: {width} != {width_scenario} or {height} != {height_scenario}")
                failed_layouts.append(layout_name)
                break
            dataset = fire['dataset']
            transformer = fire['transformer']
            x_fire, y_fire = transformer.transform(fire['LONGITUDE'], fire['LATITUDE'], direction='INVERSE')
            row, col = rasterio.transform.rowcol(dataset.transform, x_fire, y_fire)
            # print("row, col", row, col)
            ignition_point = (col, row)
            sample, sampled_ignition_point = sampler.get_scenario_location(ignition_point, leeway_distance=5, sampling_method='closest', exclude_scenarios=sampled_scenarios)
            if sample is None:
                failed += 1
                continue
            sampled_scenarios.append(sample)
            sampled_ignition_points.append(sampled_ignition_point)
            associated_fires.append(fire['OBJECTID'])
            distance = abs(sampled_ignition_point[0] - ignition_point[0]) + abs(sampled_ignition_point[1] - ignition_point[1])
            distances.append(distance)
        # plot the sampled scenarios
        # the axes are inverted as coordinates start in (0,0) in the top left corner
        fig, ax = plt.subplots()
        ax.scatter([point[0] for point in sampled_ignition_points], [width - point[1] for point in sampled_ignition_points], color='red')
        plt.show()
        print(f"Failed {failed}")
        # write the selected scenarios in a txt file
        with open(f"./WideDataset/{layout_name}/selected_scenarios.txt", "w") as f:
            for scenario, fire_id in zip(sampled_scenarios, associated_fires):
                f.write(f"{scenario}, {fire_id}\n")
            f.write(f"Failed: {failed}\n")
            failed_percentage = failed / max(len(data),1)
            f.write(f"Failed percentage: {failed_percentage}\n")
        if len(data) == 0 or failed_percentage > 0.2:
            print(f"!! Failed {failed} out of {len(data)} for layout {layout_name} !!")
            failed_layouts.append(layout_name)
    except Exception as e:
        print(f"Error for layout {layout_name}: {e}")
        failed_layouts.append(layout_name)
print(processed)
print(total)
    

# TODO do we need to create the grid manually? I think the raster file will doirectly give you the coordinates within the layout


# for each fire, sample the scenario (space only, and time+space)
# write the identifier in a txt file (one for space only, one for time+space)

# move the scenarios into a selected folder
# "delete" the other scenarios

# train test split with the date












FileNotFoundError: [Errno 2] No such file or directory: './WideDataset/'

# same with date

In [ ]:
import geopandas as gpd

# # Path to your GeoJSON file
# geojson_file = './National_USFS_Fire_Occurrence_Point_(Feature_Layer).geojson'

# # Read the GeoJSON file
# gdf = gpd.read_file(geojson_file)

# # Path to the output GeoPackage file
# gpkg_file = 'newfires.gpkg'

# # Save the GeoDataFrame to a GeoPackage file
# gdf.to_file(gpkg_file, driver='GPKG')



KeyboardInterrupt



In [4]:
new_fires_gdf = gpd.read_file("./newfires.gpkg")

In [6]:
# remove all lines with DISCOVERYDATETIME = 1001/01/01 00:00:00+00
print(len(new_fires_gdf))
new_fires_gdf = new_fires_gdf[new_fires_gdf['DISCOVERYDATETIME'] != '1001/01/01 00:00:00+00']
print(len(new_fires_gdf))

582291
581759


In [7]:
new_fires_gdf['DISCOVERY_DATE'] = pd.to_datetime(new_fires_gdf['DISCOVERYDATETIME'])
new_fires_gdf['DISCOVERY_DATE'].dt.date


0         2014-07-03
1         2014-06-06
2         2014-07-28
3         2014-08-19
4         2014-08-26
             ...    
582286    2005-09-11
582287    2010-04-08
582288    2009-09-29
582289    1996-07-06
582290    2021-07-14
Name: DISCOVERY_DATE, Length: 581759, dtype: object

In [8]:
print(new_fires_gdf['geometry'].iloc[1])
print(new_fires_gdf['LATDD83'].iloc[1], new_fires_gdf['LONGDD83'].iloc[1])


POINT (-107.24360000079399 36.97472000088236)
36.97471 -107.24359


In [9]:
new_fires_gdf['DISCOVERY_DATE'] = pd.to_datetime(new_fires_gdf['DISCOVERYDATETIME'])
new_fires_gdf['DISCOVERY_DATE'].dt.date
new_fires_gdf['LONGITUDE'] = new_fires_gdf['LONGDD83']
new_fires_gdf['LATITUDE'] = new_fires_gdf['LATDD83']

In [10]:
# keep only the fires that occured after 2019
new_fires_gdf = new_fires_gdf[new_fires_gdf['DISCOVERY_DATE'] >= '2019-01-01']
print(len(new_fires_gdf))
new_fires_gdf['DISCOVERY_DATE'].head()


34889


24    2019-11-01 14:30:00+00:00
266   2019-07-25 15:37:00+00:00
267   2019-09-06 16:46:00+00:00
347   2022-04-20 21:56:00+00:00
348   2022-06-12 22:24:00+00:00
Name: DISCOVERY_DATE, dtype: datetime64[ns, UTC]

In [9]:
from Scenario_sampler import ScenarioSamplerDate
from tqdm import tqdm
# Get the layout coordinates 
# 1 extract the tif files
dataset_path = "./WideDataset/"

# copy the layout tifs from a folder to a folder called sim2real_layout
tif_path = "sim2real_layouts"
os.makedirs(tif_path, exist_ok=True)
for folder in os.listdir(dataset_path):
    if folder == ".DS_Store":
        continue
    shutil.copy(f"WideDataset/{folder}/Vegetation_Map/Existing_Vegetation_Cover.tif", f"{tif_path}/{folder}_Existing_Vegetation_Cover.tif")

print("Copied the layout tifs to sim2real_layouts")
####

tif_files = glob(os.path.join(tif_path, "*.tif"))
layout_list = []
widths = []
names = []
for tif_file in tif_files:
    with rasterio.open(tif_file) as dataset:
        # get the file name without the path
        file_name = os.path.basename(tif_file)  # '0004_Elevation.tif'
        identifier = "_".join(file_name.split('_')[:2])

        # extract the resolution, check it is 30
        x_resolution = dataset.transform[0]
        y_resolution = -dataset.transform[4]
        assert x_resolution == y_resolution == 30, f"Resolution is not the same: {x_resolution} != {y_resolution}"

        # extract the coordinates using the bounds
        # /!\ DO NOT USE transform_bounds(dataset.crs, 'EPSG:4326', *dataset.bounds)
        x_min = dataset.bounds[0]
        x_max = dataset.bounds[2]
        y_min = dataset.bounds[1]
        y_max = dataset.bounds[3]

        transformer = Transformer.from_crs(dataset.crs, "EPSG:4326", always_xy=True)
        lat_top_left, lon_top_left = transformer.transform(x_min, y_max)
        lat_top_right, lon_top_right = transformer.transform(x_max, y_max)
        lat_bottom_left, lon_bottom_left = transformer.transform(x_min, y_min)
        lat_bottom_right, lon_bottom_right = transformer.transform(x_max, y_min)

        # Create the polygon using the transformed bounds
        polygon = Polygon((
            (lat_top_left, lon_top_left),
            (lat_top_right, lon_top_right),
            (lat_bottom_right, lon_bottom_right),
            (lat_bottom_left, lon_bottom_left),
            (lat_top_left, lon_top_left)  # close the polygon
        ))

        layout_list.append({
            'identifier': identifier,
            'height': dataset.height,
            'width': dataset.width,
            'geometry': polygon,
            'transformer': transformer,
            'dataset': dataset
        })


        widths.append(dataset.width)
        names.append(identifier)

sorted_indices = np.argsort(widths)
widths = np.array(widths)[sorted_indices]
names = np.array(names)[sorted_indices]
filtered_layout_list = [layout_list[i] for i in sorted_indices]

n_small_layouts = len(widths[widths < 500])
n_medium_layouts = len(widths[(widths >= 500) & (widths < 1000)])
n_large_layouts = len(widths[widths >= 1000])

small_layouts = filtered_layout_list[:n_small_layouts]
medium_layouts = filtered_layout_list[n_small_layouts:n_small_layouts + n_medium_layouts]
large_layouts = filtered_layout_list[n_small_layouts + n_medium_layouts:]

print("loaded the layout list")


# convert the layout list to a geopandas df
gdf = gpd.GeoDataFrame(layout_list, geometry='geometry', crs="EPSG:4326")




# Load the historical fires 
# fires_gdf = gpd.read_file("./FPA_FOD_20210617.gpkg")
# fires_gdf = fires_gdf.to_crs("EPSG:4326")
print("loaded the fires")


# Joint
# Spatial join: find which points fall into which polygons
joined = gpd.sjoin(new_fires_gdf, gdf, how='inner', predicate='within')


# Count points per polygon
counts = joined.groupby('identifier').size().reset_index(name='fire_count')
print(counts)


dataset_path = "./WideDataset/"
scenario_path_suffix = "/Satellite_Images_Mask/"

failed_layouts = []
processed_layouts = []
continue_out = False
processed = total = 0
WORKED = 0

for layout_name in tqdm(names):
    total+=1
    #print(layout_name)
    try:
        layout_folder = dataset_path + layout_name + scenario_path_suffix
        # check the layout folder exists
        if not os.path.exists(layout_folder):
            print(f"Layout {layout_folder} does not exist")
            layout_folder = dataset_path + layout_name +  "/Satellite_Image_Mask/"
            if not os.path.exists(layout_folder):
                print(f"Layout {layout_folder} does not exist")
                failed_layouts.append(layout_name)
                continue
        
        if os.path.exists(f"./WideDataset/{layout_name}/selected_scenarios.txt"):
            processed +=1
            #continue

        # check that the scenario have the right size
        first_scenario = return_first_scenario(layout_folder)
        if first_scenario is None:
            print(f"Layout {layout_name} does not have any scenario")
            failed_layouts.append(layout_name)
            continue
        
        first_loaded_scenario = load_scenario(os.path.join(layout_folder, first_scenario), extension = '.jpg', first_frame_only=True)
        height_scenario, width_scenario = first_loaded_scenario.shape[0], first_loaded_scenario.shape[1]

        earliest_date, latest_date = find_earliest_latest_dates(dataset_path + layout_name)

        # if earliest_date.year > 2020:
        #     print(f"Layout {layout_name} has no fires before 2021: {earliest_date}")
        #     failed_layouts.append(layout_name)
        #     continue

        # plot the historical fires
        data = joined[joined['identifier'] == layout_name]

        filtered_data = data[
        (data['DISCOVERY_DATE'].dt.date >= earliest_date.date()) & 
        (data['DISCOVERY_DATE'].dt.date <= latest_date.date())
        ]
        n_data = len(filtered_data)
        if n_data == 0:
            #print(f"No fires found for layout {layout_name} between {earliest_date} and {latest_date}")
            # if len(data) != 0:  
            #     print("example  fire date:", data['DISCOVERY_DATE'].iloc[0])
            failed_layouts.append(layout_name)
            continue


        sampler = ScenarioSamplerDate(layout_folder, extension = '.jpg')
        sampled_scenarios = []
        sampled_ignition_points = []
        sampled_ignition_dates = []
        associated_fires = []
        date_matched = []
        distances = []
        failed = 0
        

        #plt.scatter(filtered_data["LONGITUDE"], filtered_data["LATITUDE"])
        #plt.axis("equal")
        #plt.show()

        # start with the fires that have potewntial to be test fires, i.e their date is between the earliest and latest date
        # We will have one "test" dataset, one "train" dataset, and one extra train dataset for the test fires

        

        for i, fire in filtered_data.iterrows():
            # print the coordinates of the fire
            width, height = fire['width'], fire['height']
            # check that the scenario have the right size
            if width != width_scenario or height != height_scenario:
                print(f"Scenario {first_scenario} has the wrong size for layout {layout_name}: {width} != {width_scenario} or {height} != {height_scenario}")
                failed_layouts.append(layout_name)
                n_data = 0 
                break
            dataset = fire['dataset']
            transformer = fire['transformer']
            x_fire, y_fire = transformer.transform(fire['LONGITUDE'], fire['LATITUDE'], direction='INVERSE')
            row, col = rasterio.transform.rowcol(dataset.transform, x_fire, y_fire)
            date = fire['DISCOVERY_DATE']
            # print("row, col", row, col)
            ignition_point = (row, col)
            res = sampler.get_scenario_location(ignition_point, date = fire['DISCOVERY_DATE'].date(), leeway_distance=10, leeway_date=1, sampling_method='closest', exclude_scenarios=sampled_scenarios)
            if res[0] is None:
                sample, sampled_ignition_point, sampled_ignition_date = None, None, None
            else:
                sample, sampled_ignition_point, sampled_ignition_date = res
            if sample is None:
                failed += 1
                print("we wanted to sample the fire", fire['DISCOVERY_DATE'], "at", ignition_point, "but it failed")
                #print(sampler.get_all_scenario_dates())
                #print(sampler.get_all_scenario_ignition_points())
                #print("-- surprising? --")
                #print(sampler.get_scenarios_at_a_date(fire['DISCOVERY_DATE'].date()))
                continue
            else:
                print(sample)
            sampled_scenarios.append(sample)
            sampled_ignition_points.append(sampled_ignition_point)
            sampled_ignition_dates.append(sampled_ignition_date)
            associated_fires.append(fire['OBJECTID'])
            distance = abs(sampled_ignition_point[0] - ignition_point[0]) + abs(sampled_ignition_point[1] - ignition_point[1])
            distances.append(distance)
        # plot the sampled scenarios
        # the axes are inverted as coordinates start in (0,0) in the top left corner
        #fig, ax = plt.subplots()
        #ax.scatter([point[0] for point in sampled_ignition_points], [width - point[1] for point in sampled_ignition_points], color='red')
        #plt.show()
        #print(f"Failed {failed}")
        # write the selected scenarios in a txt file
        with open(f"./WideDataset/{layout_name}/selected_scenarios_historical.txt", "w") as f:
            for scenario, fire_id in zip(sampled_scenarios, associated_fires):
                f.write(f"{scenario}, {fire_id}\n")
            f.write(f"Failed: {failed}\n")
            failed_percentage = failed / max(n_data,1)
            f.write(f"Failed percentage: {failed_percentage}\n")
            #print(f"Layout {layout_name}: {n_data} fires, {failed} failed, {failed_percentage}")
            processed_layouts.append(layout_name)
        if n_data != 0 and failed != n_data:
            WORKED += 1
            print(f"WORKED: {WORKED}, {sampled_scenarios}")
            print("worked: ", layout_name)
            plt.scatter(filtered_data["LONGITUDE"], filtered_data["LATITUDE"])
            plt.axis("equal")
            plt.show()
            fig, ax = plt.subplots()
            ax.scatter([point[0] for point in sampled_ignition_points], [width - point[1] for point in sampled_ignition_points], color='red')
            plt.show()
        if n_data != 0 and failed_percentage > 0.2:
            print(f"!! Failed {failed} out of {n_data} for layout {layout_name} !!")
            failed_layouts.append(layout_name)
    except Exception as e:
        print(f"Error for layout {layout_name}: {e}")
        failed_layouts.append(layout_name)
print(processed)
print(total)
print(processed_layouts)
    

# TODO do we need to create the grid manually? I think the raster file will doirectly give you the coordinates within the layout


# for each fire, sample the scenario (space only, and time+space)
# write the identifier in a txt file (one for space only, one for time+space)

# move the scenarios into a selected folder
# "delete" the other scenarios

# train test split with the date









Copied the layout tifs to sim2real_layouts
loaded the layout list
loaded the fires
    identifier  fire_count
0   0016_03070         192
1   0037_01578          21
2   0041_02386          66
3   0057_03186         279
4   0058_03866         183
5   0059_02804         264
6   0060_03010         381
7   0061_03726         328
8   0062_03187          55
9   0063_02387         896
10  0064_02717         172
11  0065_03061         116
12  0066_03773         174
13  0067_03550         280
14  0068_04211         142
15  0069_03539         629
16  0081_03471         252
17  0082_03155         691
18  0083_02892        1098
19  0084_02609         249
20  0090_00987         537
21  0092_03189          26
22  0093_01748           1
23  0094_01688           1
24  0095_01726           1
25  0098_01784         544
26  0100_02449         544
27  0102_01733         714
28  0103_01810         714
29  0104_02422         180
30  0105_03054         142
31  0106_02165          43
32  0111_03612          59

  7%|▋         | 5/72 [00:00<00:01, 35.65it/s]

Error for layout 0344_03155: No subfolder found
Layout ./WideDataset/0106_02165/Satellite_Images_Mask/ does not exist
Layout ./WideDataset/0111_03612/Satellite_Images_Mask/ does not exist


 12%|█▎        | 9/72 [00:00<00:02, 22.97it/s]

Layout ./WideDataset/0104_02422/Satellite_Images_Mask/ does not exist
Error for layout 0089_00984: No subfolder found
we wanted to sample the fire 2023-07-24 23:34:00+00:00 at (396, 62) but it failed
we wanted to sample the fire 2023-07-24 23:34:00+00:00 at (396, 62) but it failed
!! Failed 2 out of 2 for layout 0016_03070 !!


...

Error for layout 0083_02892: unconverted data remains:  
Error for layout 0021_01232: No subfolder found
Error for layout 0020_00970: No subfolder found
Layout ./WideDataset/0063_02387/Satellite_Images_Mask/ does not exist


 81%|████████  | 58/72 [00:07<00:01,  8.34it/s]

Error for layout 0063_02387: unconverted data remains:  
Layout ./WideDataset/0061_03726/Satellite_Images_Mask/ does not exist


100%|██████████| 72/72 [00:07<00:00,  9.54it/s]

Error for layout 0098_01784: No subfolder found
Error for layout 0100_02449: No subfolder found
Error for layout 0113_03495: No subfolder found
Error for layout 0114_02292: No subfolder found
Error for layout 0257_02175: No subfolder found
Error for layout 0258_02858: No subfolder found
Error for layout 0255_02103: No subfolder found
Error for layout 0254_02361: No subfolder found
Error for layout 0256_02752: No subfolder found
Error for layout 0253_03246: No subfolder found
71
72
['0016_03070']


In [28]:
print(len(new_fires_gdf['OBJECTID'].unique()))
print(len(new_fires_gdf))

581759
581759


In [ ]:
with open()